In [16]:
 import pandas as pd

In [17]:
df = pd.read_csv('data/2791_2019dar.csv')

In [18]:
# Display a list of available teams
team_nums = df['Team Number']
team_nums = team_nums.drop_duplicates()

pd.set_option('display.max_rows', None)
print('Available Teams:')
print(team_nums)
pd.reset_option('display.max_rows')

Available Teams:
0       51
1      868
2     3044
3     4381
4     3381
5     7651
6      245
7     3604
8     1792
9     5951
10    1690
11    6401
12    5501
13     503
14    3618
15    3555
16    2500
17     125
18     250
19    3624
20    1923
21    6806
22    2337
23    1317
24     836
25    3926
26     236
27     103
28     225
29    1918
30    2791
31    4253
32    2502
33    5348
34    7480
35    5943
36    4481
37    5567
38    5811
39    2771
40     217
41    7885
42    1058
43    3707
44     365
45     771
46    2830
47     190
48    3146
49    1218
50    3542
51     449
52    7462
53    7719
54    5084
55     191
56      88
57    1676
58     175
59    3667
60     772
61    4143
62    1024
63    7673
64    2265
65     353
66      16
70    1781
Name: Team Number, dtype: int64


In [19]:
# Calculate the average score of a specified team
# Parameter team_num: the team number of the team to find the avg score of

def avg_score(team_num):

    # Creates a new dataframe containing only data for team team_num
    teamdf = df[df['Team Number'] == team_num]

    total_score = 0

    for i in teamdf.index:

        # Hab Line Cross
        if teamdf['Hab Line 1/0'][i] == 1:
            startpos = teamdf['Starting Position'][i]
            if startpos == 'L1' or startpos == 'M' or startpos == 'R1':
                total_score += 3
            elif startpos == 'L2' or startpos == 'R2':
                total_score += 6

        # Hatch Panels
        total_score += 2 * (teamdf['Auto # H Ship Side'][i] + teamdf['Auto # H Ship Front'][i] +
                        teamdf['Tele H Ship'][i] + teamdf['Tele Rocket H L1'][i] + teamdf['Tele Rocket H L2'][i] +
                        teamdf['Tele Rocket H L3'][i])

        if teamdf['Auto H Rkt Lvl'][i] > 0:
            total_score += 2

        if teamdf['Auto H Rkt Lvl [2]'][i] > 0:
            total_score += 2

        # Cargo
        total_score += 3 * (teamdf['Auto # C Ship Side'][i] + teamdf['Tele C Ship'][i] + teamdf['Tele Rocket C L1'][i] +
                        teamdf['Tele Rocket C L2'][i] + teamdf['Tele Rocket C L3'][i])

        if teamdf['Auto C Rkt Lvl'][i] > 0:
            total_score += 3

        # Fouls
        if teamdf['Fouls? 0/1'][i] == 1:
            total_score -= 7

        # Hab Climb Bonus
        level = teamdf['Highest success'][i]
        if level == 1:
            total_score += 3
        elif level == 2:
            total_score += 6
        elif level == 3:
            total_score += 12

    return total_score / len(teamdf.index)

In [20]:
# Calculate a team's average score achieved during sandstorm
# Parameter team_num: the team number of the team to find the avg sandstorm score of

def sandstorm_score(team_num):

    # Creates a new dataframe containing only data for team team_num
    teamdf = df[df['Team Number'] == team_num]

    total_score = 0

    for i in teamdf.index:

        # Hab Line Cross
        if teamdf['Hab Line 1/0'][i] == 1:
            startpos = teamdf['Starting Position'][i]
            if startpos == 'L1' or startpos == 'M' or startpos == 'R1':
                total_score += 3
            elif startpos == 'L2' or startpos == 'R2':
                total_score += 6

        # Hatch Panels
        total_score += 2 * (teamdf['Auto # H Ship Side'][i] + teamdf['Auto # H Ship Front'][i])

        if teamdf['Auto H Rkt Lvl'][i] > 0:
            total_score += 2

        if teamdf['Auto H Rkt Lvl [2]'][i] > 0:
            total_score += 2

        # Cargo
        total_score += 3 * (teamdf['Auto # C Ship Side'][i])

        if teamdf['Auto C Rkt Lvl'][i] > 0:
            total_score += 3

    return total_score / len(teamdf.index)

In [21]:
# Calculate a team's average score achieved during teleop
# Parameter team_num: team number

def teleop_score(team_num):

    # Creates a new dataframe containing only data for team team_num
    teamdf = df[df['Team Number'] == team_num]

    total_score = 0

    for i in teamdf.index:

        # Hatch Panels
        total_score += 2 * (teamdf['Tele H Ship'][i] + teamdf['Tele Rocket H L1'][i] + teamdf['Tele Rocket H L2'][i] +
                            teamdf['Tele Rocket H L3'][i])

        # Cargo
        total_score += 3 * (teamdf['Tele C Ship'][i] + teamdf['Tele Rocket C L1'][i] + teamdf['Tele Rocket C L2'][i] +
                            teamdf['Tele Rocket C L3'][i])

        # Fouls
        if teamdf['Fouls? 0/1'][i] == 1:
            total_score -= 7

    return total_score / len(teamdf.index)

In [22]:
# Calculate a specified team's defense capability
# Parameter team_num: the team number of the team to find the defense capability of
# Remove this function? See 2 functions below -->
def defense_score(team_num):

    # Creates a new dataframe containing only data for team team_num
    teamdf = df[df['Team Number'] == team_num]

    defense_score = 0

    for i in teamdf.index:
        defense_score += teamdf['Def pl amt'][i] + teamdf['Def pl quality'][i]

    return defense_score / len(teamdf.index)

In [23]:
# Determine the average amount of defense played by a specified team:

def defense_amount(team_num):

    # Creates a new dataframe containing only data for team team_num
    teamdf = df[df['Team Number'] == team_num]

    defense_amount = 0

    for i in teamdf.index:
        defense_amount += teamdf['Def pl amt'][i]

    return defense_amount / len(teamdf.index)

In [24]:
# Calculate the average quality of defense played of a specified team:

def defense_quality(team_num):

    # Creates a new dataframe containing only data for team team_num
    teamdf = df[df['Team Number'] == team_num]

    total_quality = 0

    for i in teamdf.index:
        total_quality += teamdf['Def pl quality'][i]

    return total_quality / len(teamdf.index)

In [25]:
# Determine how often a specified team's bot dies

def dead_score(team_num):

    # Creates a new dataframe containing only data for team team_num
    teamdf = df[df['Team Number'] == team_num]

    dead_score = 0

    for i in teamdf.index:
        if teamdf['Dead 0-3'][i] == 1:
            dead_score += 2
        elif teamdf['Dead 0-3'][i] == 2:
            dead_score += 7
        elif teamdf['Dead 0-3'][i] == 3:
            dead_score += 15

    return dead_score / len(teamdf.index)

In [26]:
def view_dashboard(team_num):
        output = 'Team ' + str(team_num) + ' Stats:\n'

        # Average Score
        output += '\n\tAverage Score: ' + str(avg_score(team_num)) + ' points\n'

        # Average Score during Auto
        output += '\n\tAverage Score During Auto: ' + str(sandstorm_score(team_num))+ ' points\n'

        # Average Score during Teleop
        output += '\n\tAverage Score During Teleop: ' + str(teleop_score(team_num))+ ' points\n'

        # Defense Amount
        output += '\n\tDefense Amount: '+ str(defense_amount(team_num)) + '\n'

        # Defense Quality
        dq = defense_quality(team_num)
        output += '\n\tDefense Quality: '

        if dq >= 4:
            output += 'Flawless\n'
        elif dq >= 3:
            output += 'Excellent\n'
        elif dq >= 2:
            output += 'Average\n'
        elif dq >= 1:
            output += 'Bad\n'
        elif dq >= 0:
            output += 'Non-existent\n'

        # Dead Amount
        output += '\n\tAmount of time dead: '+ str(dead_score(team_num))+ '\n'

        print(output)

In [31]:
def choose_members():

    #initialize code, read in team number column only
    team_df = df['Team Number']
    team_df = team_df.drop_duplicates()

    team_scores = []
    team_sandstorm = []
    team_teleop = []
    team_defense_scores = []
    team_defense_amount = []
    team_defense_quality = []
    team_dead_scores = []

    team_df.dropna(inplace = True)

    # get average score of all teams
    for i in team_df.index:
        avg = avg_score(team_df[i])
        team_scores.append(avg)

        sandstorm = sandstorm_score(team_df[i])
        team_sandstorm.append(sandstorm)

        teleop = teleop_score(team_df[i])
        team_teleop.append(teleop)

        ds = defense_score(team_df[i])
        team_defense_scores.append(ds)

        da = defense_amount(team_df[i])
        team_defense_amount.append(da)

        dq = defense_score(team_df[i])
        team_defense_quality.append(dq)

        dsc = dead_score(team_df[i])
        team_dead_scores.append(dsc)


    
    # data frame
    d = {'Team Number': team_df, 'Average Scores': team_scores}
    d1 = {'Team Number': team_df, 'Sandstorm Scores': team_sandstorm}
    d2 = {'Team Number': team_df, 'Teleop Scores': team_teleop}
    d3 = {'Team Number': team_df, 'Defense Scores': team_defense_scores}
    d4 = {'Team Number': team_df, 'Defense Amount': team_defense_amount}
    d5 = {'Team Number': team_df, 'Defense Quality': team_defense_quality}
    d6 = {'Team Number': team_df, 'Dead Scores': team_dead_scores}

    df2 = pd.DataFrame(data = d)
    df3 = pd.DataFrame(data = d1)
    df4 = pd.DataFrame(data = d2)
    df5 = pd.DataFrame(data = d3)
    df6 = pd.DataFrame(data = d4)
    df7 = pd.DataFrame(data = d5)
    df8 = pd.DataFrame(data = d6)
    
    # natural sort using average team scores
    df2 = df2.sort_values(by=['Average Scores'])
    df3 = df3.sort_values(by=['Sandstorm Scores'])
    df4 = df4.sort_values(by=['Teleop Scores'])
    df5 = df5.sort_values(by=['Defense Scores'])
    df6 = df6.sort_values(by=['Defense Amount'])
    df7 = df7.sort_values(by=['Defense Quality'])
    df8 = df8.sort_values(by=['Dead Scores'])

    print(df2)
    print(df3)
    print(df4)
    print(df5)
    print(df6)
    print(df7)
    print(df8)


In [32]:
# UI
while True:
    choice = int(input("""1 - View analytics dashboard for a specified team. \n
                       2 - Choose alliance members by viewing lists of best scorers, etc. \n
                       3 - Exit the program. \n
                       Enter either '1', '2', or '3' to select an option:"""))

    if choice == 1:
        team_input = int(input('Enter the team number of the team you wish to analyze:'))
        while True:
            team_df = df[df['Team Number'] == team_input]
            if len(team_df.index) == 0:
                team_input = int(input('The team number you have entered is not found in the database. Enter a valid team number:'))
            else:
                break
        view_dashboard(team_input)
        break
    elif choice == 2:
        choose_members()
        break
    elif choice == 3:
        break
    else:
        print("Please enter either '1' or '2'!")

    Team Number  Average Scores
45          771        8.625000
16         2500        8.818182
41         7885        9.100000
18          250        9.666667
52         7462       10.285714
..          ...             ...
43         3707       35.083333
7          3604       35.090909
6           245       35.444444
56           88       36.888889
28          225       37.000000

[68 rows x 2 columns]
    Team Number  Sandstorm Scores
60          772          3.000000
4          3381          3.272727
27          103          3.454545
58          175          3.777778
34         7480          3.900000
..          ...               ...
9          5951          7.666667
2          3044          7.800000
10         1690          8.090909
14         3618          8.100000
21         6806          8.300000

[68 rows x 2 columns]
    Team Number  Teleop Scores
45          771      -0.625000
11         6401       0.818182
16         2500       0.909091
41         7885       1.000000
19     